Persistencia y particionado
----------------------

### Persistencia

Problema al usar un RDD varias veces:

-   Spark recomputa el RDD y sus dependencias cada vez que se ejecuta una acción
-   Muy costoso (especialmente en problemas iterativos)

Solución

-   Conservar el RDD en memoria y/o disco
-   Métodos `cache()` o `persist()`

#### Niveles de persistencia (definidos en [`pyspark.StorageLevel`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.StorageLevel))
 Nivel                | Espacio  | CPU     | Memoria/Disco   | Descripción
 :------------------: | :------: | :-----: | :-------------: | ------------------
 MEMORY_ONLY          |   Alto   |   Bajo  |     Memoria     | Guarda el RDD como un objeto Java no serializado en la JVM. Si el RDD no cabe en memoria, algunas particiones no se *cachearán* y serán recomputadas "al vuelo" cada vez que se necesiten. Nivel por defecto en Java y Scala.
 MEMORY_ONLY_SER      |   Bajo   |   Alto  |     Memoria     | Guarda el RDD como un objeto Java serializado (un *byte array* por partición). Nivel por defecto en Python, usando [`pickle`](http://docs.python.org/2/library/pickle.html).
 MEMORY_AND_DISK      |   Alto   |   Medio |     Ambos       | Guarda el RDD como un objeto Java no serializado en la JVM. Si el RDD no cabe en memoria, las particiones que no quepan se guardan en disco y se leen del mismo cada vez que se necesiten
 MEMORY_AND_DISK_SER  |   Bajo   |   Alto  |     Ambos       | Similar a MEMORY_AND_DISK pero usando objetos serializados.
 DISK_ONLY            |   Bajo   |   Alto  |     Disco       | Guarda las particiones del RDD solo en disco.
 OFF_HEAP             |   Bajo   |   Alto  |   Memoria       | Guarda el RDD serializado usando memoria *off-heap* (fuera del heap de la JVM) lo que puede reducir el overhead del recolector de basura
   


    
#### Nivel de persistencia

-   En Scala y Java, el nivel por defecto es MEMORY\_ONLY

-   En Python, los datos siempre se serializan (por defecto como objetos *pickled*)

    -   Los niveles MEMORY_ONLY, MEMORY_AND_DISK son equivalentes a MEMORY_ONLY_SER, MEMORY_AND_DISK_SER
    - Es posible especificar serialización [`marshal`](https://docs.python.org/2/library/marshal.html#module-marshal) al crear el SparkContext
    
```python
sc = SparkContext(master="local", appName="Mi app", serializer=pyspark.MarshalSerializer())
```
    
#### Recuperación de fallos

-   Si falla un nodo con datos almacenados, el RDD se recomputa

    -   Añadiendo `_2` al nivel de persistencia, se guardan 2 copias del RDD
        
#### Gestión de la cache

-   Algoritmo LRU para gestionar la cache

    -   Para niveles *solo memoria*, los RDDs viejos se eliminan y se recalculan
    -   Para niveles *memoria y disco*, las particiones que no caben se escriben a disco


In [5]:
rdd = sc.parallelize(range(1000), 10)

print(rdd.is_cached)

In [6]:
rdd.persist(StorageLevel.MEMORY_AND_DISK_SER_2)

print(rdd.is_cached)

print("Nivel de persistencia de rdd: {0} ".format(rdd.getStorageLevel()))

In [7]:
rdd2 = rdd.map(lambda x: x*x)
print(rdd2.is_cached)


In [8]:
rdd2.cache() # Nivel por defecto
print(rdd2.is_cached)
print("Nivel de persistencia de rdd2: {0}".format(rdd2.getStorageLevel()))


In [9]:
rdd2.unpersist() # Sacamos rdd2 de la cache
print(rdd2.is_cached)

### Particionado

El número de particiones es función del tamaño del cluster o el número de bloques del fichero en HDFS

-   Es posible ajustarlo al crear u operar sobre un RDD

-   El paralelismo de RDDs que derivan de otros depende del de sus RDDs padre

-   Dos funciones útiles:

    -   `rdd.getNumPartitions()` devuelve el número de particiones del RDD
    -   `rdd.glom()` devuelve un nuevo RDD juntando los elementos de cada partición en una lista


In [11]:
rdd = sc.parallelize([1, 2, 3, 4, 2, 4, 1], 4)
pairs = rdd.map(lambda x: (x, x))

print("RDD pairs = {0}".format(
        pairs.collect()))
print("Particionado de pairs: {0}".format(
        pairs.glom().collect()))
print("Número de particiones de pairs = {0}".format(
        pairs.getNumPartitions()))

In [12]:
# Reducción manteniendo el número de particiones
print("Reducción con 4 particiones: {0}".format(
        pairs.reduceByKey(lambda x, y: x+y).glom().collect()))

In [13]:
# Reducción modificando el número de particiones
print("Reducción con 2 particiones: {0}".format(
       pairs.reduceByKey(lambda x, y: x+y, 2).glom().collect()))

#### Funciones de reparticionado
- `repartition(n)` devuelve un nuevo RDD que tiene exactamente `n` particiones
- `coalesce(n)` más eficiente que `repartition`, minimiza el movimiento de datos
    - Solo permite reducir el número de particiones
- `partitionBy(n,[partitionFunc])` Particiona por clave, usando una función de particionado (por defecto, un hash de la clave)
    - Solo para RDDs clave/valor
    - Asegura que los pares con la misma clave vayan a la misma partición


In [15]:
pairs5 = pairs.repartition(5)
print("pairs5 con {0} particiones: {1}".format(
        pairs5.getNumPartitions(),
        pairs5.glom().collect()))

In [16]:
pairs2 = pairs5.coalesce(2)
print("pairs2 con {0} particiones: {1}".format(
        pairs2.getNumPartitions(),
        pairs2.glom().collect()))


In [17]:
pairs_clave = pairs2.partitionBy(3)
print("Particionado por clave ({0} particiones): {1}".format(
        pairs_clave.getNumPartitions(),
        pairs_clave.glom().collect())) 